# W tym notebooku przekształcane są wartości z każdego wiersza każdej kolumny na wartości pobrane z dbpedii

In [1]:
import pandas as pd

## Sprawdzenie postaci mapowań tekst -> klasy dbpedii

In [2]:
df = pd.read_csv("items_classes_df.csv")

In [3]:
df.head()

,item,classes
0,The_Godfather,"""http://dbpedia.org/ontology/Film"";""http://dbp..."
1,Citizen_Kane,"""http://dbpedia.org/ontology/Film"";""http://dbp..."
2,Casablanca,"""http://dbpedia.org/ontology/Place"";""http://db..."
3,Lawrence_of_Arabia,"""Lawrence_of_Arabia"""
4,Dr_Strangelove_or_How_I_Learned_to_Stop_Worryi...,"""Dr_Strangelove_or_How_I_Learned_to_Stop_Worry..."


In [4]:
df = df.fillna("")

## Zapisanie danych z dataframe do słownika dla szybkiego dostępu

In [5]:
classes_translated = {}
for cc, row in df.iterrows():
    classes_translated[row["item"]] = row["classes"]

In [6]:
len(classes_translated)

8669

## Wczytanie bazowego datasetu

In [7]:
dane = pd.read_csv("glove_df_p.csv")

In [8]:
dane

,csv,text,http://dbpedia.org/ontology/Plant,http://dbpedia.org/ontology/PopulatedPlace,http://dbpedia.org/ontology/EducationalInstitution,http://dbpedia.org/ontology/Writer,http://dbpedia.org/ontology/TelevisionShow,http://dbpedia.org/ontology/Museum,http://dbpedia.org/ontology/Lake,http://dbpedia.org/ontology/Person,...,http://dbpedia.org/ontology/BaseballPlayer,http://dbpedia.org/ontology/Company,http://dbpedia.org/ontology/Newspaper,http://dbpedia.org/ontology/Mountain,http://dbpedia.org/ontology/Bird,http://dbpedia.org/ontology/Wrestler,http://dbpedia.org/ontology/Animal,http://dbpedia.org/ontology/Film,http://dbpedia.org/ontology/Eukaryote,http://dbpedia.org/ontology/City
0,58891288_0_1117541047012405958,"""The Godfather"";""Citizen Kane"";""Casablanca"";""L...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,8468806_0_4382447409703007384,"""Kaspisches-Meer"";""Oberer-See"";""Victoriasee"";""...",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,50245608_0_871275842592178099,"""10 Things I Hate About You"";""12 Angry Men"";""2...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,14067031_0_559833072073397908,"""Arabic"";""Portuguese"";""French"";""English"";""Fren...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8286121_0_8471791395229161598,"""South America Continent"";""Saint Helena"";""Sain...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,66009064_0_9148652238372261251,"""Astros"";""Mariners"";""Diamondbacks"";""Athletics""...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,21362676_0_6854186738074119688,"""The Game"";""Gandhi"";""Gattaca"";""The General"";""G...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
117,39107734_2_2329160387535788734,"""Atlanta, Georgia, United States"";""Hillingdon,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
118,29414811_13_8724394428539174350,"""Batman: Arkham City[131]"";""Super Mario Galaxy...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
dane = dane.fillna("")

## Funkcja mapująca wartości tekstowe na wartości z dbpedii

In [14]:
def process_data(text, classes_translated):
    splt = text.split('";"')
    toret = []
    for item in splt:
        item = item.replace('"', "")\
            .replace(" ","_")\
            .replace(".", "")\
            .replace(":", "")\
            .replace("'", "")\
            .replace("-", "")\
            .replace("/", "")\
            .replace("?", "")\
            .replace("*", "")\
            .replace(",", "")\
            .replace("&", "")\
            .replace("(", "")\
            .replace(")", "")\
            .replace("!", "")\
            .replace("[", "")\
            .replace("]", "")\
            .replace("=", "")\
            .replace(">", "")\
            .replace("+", "")\
            .replace(";", "")\
            .replace('"', "")\
            .replace("__", "_")
        if item == "nan":
            newitem = item.replace('";"', '" ; "').replace("_", " ")
        if item not in classes_translated.keys():
            newitem = item.replace('";"', '" ; "').replace("_", " ")
        else:
            newitem = classes_translated[item].replace('";"', '" ; "')
            newitem = newitem.replace("http://dbpedia.org/ontology/", "").replace("_", " ")
        toret.append(newitem)
        
    return toret

In [15]:
process_data('"South_America_Continent";"The Godfather";"Citizen Kane";"Casablanca"', classes_translated)

['South America Continent',
 '"Film" ; "Wikidata:Q11424" ; "Work"',
 '"Film" ; "Wikidata:Q11424" ; "Work"',
 '"Place" ; "Location" ; "City" ; "PopulatedPlace" ; "Settlement"']

In [16]:
dane["text_dbpedied"] = dane["text"].apply(lambda x: " ; ".join(process_data(x, classes_translated)))

In [17]:
dane["text_dbpedied"]

0      "Film" ; "Wikidata:Q11424" ; "Work" ; "Film" ;...
1      "KaspischesMeer" ; "ObererSee" ; "Victoriasee"...
2      "Film" ; "Wikidata:Q11424" ; "Work" ; "12 Angr...
3      "Language" ; "Portuguese" ; "French" ; "Englis...
4      South America Continent ; "Place" ; "Location"...
                             ...                        
115    "Astros" ; "Mariners" ; "Diamondbacks" ; "Athl...
116    "The Game" ; "Gandhi" ; "Film" ; "Wikidata:Q11...
117    "Atlanta Georgia United States" ; "Hillingdon ...
118    "Batman Arkham City131" ; "Super Mario Galaxy ...
119    "Software" ; "VideoGame" ; "Work" ; "Software"...
Name: text_dbpedied, Length: 120, dtype: object

## Zapis

In [18]:
dane.to_csv("glove_df_dbpedied_p.csv")